<a href="https://colab.research.google.com/github/illaho/Master/blob/master/piste_ciclabili.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [15]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import json
import pprint
import time
import pprint

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
wd_home = webdriver.Chrome('chromedriver', options=options)
wd_home.get("https://www.piste-ciclabili.com/piste_ciclabili/") #homepage
wd = webdriver.Chrome('chromedriver', options=options) #per le sotto pagine

In [ ]:
wd_home.save_screenshot('screenshot.png')
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/content/screenshot.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(wd_home.title)

In [ ]:
elenco_regioni = wd_home.find_elements_by_css_selector("ul.home-region li")
print(len(elenco_regioni))
print(elenco_regioni)

In [ ]:
lista_regioni = []
for num in range(len(elenco_regioni)):
    nome_regione = wd_home.find_elements_by_css_selector("ul.home-region li a")[num].text
    link_sottopagine_regione = wd_home.find_elements_by_css_selector("ul.home-region li a")[num].get_attribute("href")
    lista_regioni.append({'_id': num,
                          'nome_regione': nome_regione,
                          'link_sottopagine_regione': link_sottopagine_regione}) 
print(len(lista_regioni))
pprint.pprint(lista_regioni)   

In [ ]:
ds_regioni = pd.DataFrame(lista_regioni)
ds_regioni.set_index("_id")
ds_regioni.info()

In [ ]:
ds_regioni.to_csv('/ds_regioni.csv')
ds_regioni = pd.read_csv('/ds_regioni.csv')

In [ ]:
dettagli_percorsi = []
for num in tqdm(range(len(elenco_regioni))):
    link_sottopagine_regione = wd_home.find_elements_by_css_selector("ul.home-region li a")[num].get_attribute("href")
    time.sleep(3 + np.random.randn()*0.2)
    wd.get(link_sottopagine_regione)
    #wd.save_screenshot(f'screenshot_{nome_regione}.png') --per verificare 
    lista_percorsi = wd.find_elements_by_css_selector("#tableItinerariBody tr")
    for percorso in tqdm(lista_percorsi):
        rif_regione = num
        percorso_id = percorso.find_element_by_css_selector("td a").get_attribute("href").split("/")[4].split("-")[0]
        nome_percorso = percorso.find_element_by_css_selector("td a").text
        url = percorso.find_element_by_css_selector("td a").get_attribute("href")
        distanza = percorso.find_elements_by_css_selector("td")[1].text
        tipo = percorso.find_elements_by_css_selector("td")[2].text
        fondo = percorso.find_elements_by_css_selector("td")[3].text
        pendenza_max = percorso.find_elements_by_css_selector("td")[4].text
        dislivello = percorso.find_elements_by_css_selector("td")[5].text
        dettagli_percorsi.append({'_id': percorso_id,
                                  'nome_percorso': nome_percorso,
                                  'url' : url,
                                  'distanza' : distanza,
                                  'tipo' : tipo,
                                  'fondo' : fondo,
                                  'pendenza_max' : pendenza_max,
                                  'dislivello' : dislivello,
                                  'rif_regione': rif_regione})
print(len(dettagli_percorsi))
pprint.pprint(dettagli_percorsi)

In [ ]:
ds_dettagli_percorsi = pd.DataFrame(dettagli_percorsi)
ds_dettagli_percorsi.set_index("_id")
ds_dettagli_percorsi.head()

In [ ]:
ds_dettagli_percorsi.info()

In [ ]:
#ds_dettagli_percorsi.to_csv('/ds_dettagli_percorsi.csv')
ds_dettagli_percorsi = pd.read_csv('/content/ds_dettagli_percorsi.csv')

In [ ]:
import requests
info_aggiuntive = []
lista_commenti = []
elenco_luoghi_attraversati = []
elenco_img_altimetrie = []
for _id, percorso in tqdm(ds_dettagli_percorsi.loc[2001:].iterrows()):  #.loc[2000:] eseguito scraping in 2parti
    #time.sleep(1 + np.random.randn()*0.2)
    link = percorso["url"]
    percorso_id = link.split("/")[4].split("-")[0]
    wd.get(link)
    #info_aggiuntive
    bambini = wd.find_elements_by_css_selector("table.dettagli_itinerario tbody tr")[4].text.split(":")[-1]
    pattinatori = wd.find_elements_by_css_selector("table.dettagli_itinerario tbody tr")[5].text.split(":")[-1]
    descrizione = wd.find_element_by_css_selector("#container .main").text.split("stampabile")[-1].split("Itinerario segnalato")[0]
    stringa_ultimo_aggiornam = wd.find_element_by_css_selector("#container .main").text.split("stampabile")[-1].split("Itinerario segnalato")[-1].split("Commenta")[0]
    if "il" in stringa_ultimo_aggiornam:
        ultimo_aggiornam = stringa_ultimo_aggiornam.split("il")[-1] #es il 24 giugno 2020
    else:
        ultimo_aggiornam = stringa_ultimo_aggiornam.split("l'")[-1] #es l'11 settembre 2020
    link_imm_percorso = wd.find_element_by_css_selector("h2.roadbook-link a").get_attribute("href")
    info_aggiuntive.append({'_id': percorso_id,
                            'bambini': bambini,
                            'pattinatori': pattinatori,
                            'descrizione': descrizione,
                            'ultimo_aggiornam': ultimo_aggiornam,
                            'link_imm_percorso': link_imm_percorso})
    #immagini_altimetrie
    src = wd.find_element_by_css_selector("div.imageContainer img.altimetricImage").get_attribute("src")
    elenco_img_altimetrie.append({"_id": _id, #non ho assegnato bene l'id (percorso_id), devo rinominare i file...vedi cella più sotto
                                  "img_file": "altimetria_" + str(_id) + ".jpg"}) #idem _id
    img_file = requests.get(src, stream=True)
    if img_file.status_code == 200: #prima salva cartella!!!!!
      with open("/content/images_percorsi_altimetrie/altimetria_" + str(_id) + ".jpg", 'wb') as f: #idem _id 
          f.write(img_file.content) 
    #lista_commenti
    try:
      commenti = wd.find_element_by_css_selector(".comments h2 a").text
      if "(" in commenti: #se sono presenti commenti il numero degli stessi è contenuto tra parentesi, es: Commenti (5)
          num_commenti_stringa = commenti.split("(")[-1].split(")")[0]
          num_commenti = int(num_commenti_stringa)
          for num in range(num_commenti):
              commento = wd.find_elements_by_css_selector(".comments ul li")[num].text.split('\n')[0]
              data = wd.find_elements_by_css_selector(".comments ul li")[num].text.split(',')[-1]
              #commento = container_commenti[num_commento].text
              lista_commenti.append({'_id': percorso_id,
                                    'commento': commento,
                                    'data': data})
    except:
        lista_commenti.append({'_id': percorso_id,
                              'commento': "N/D",
                               'data': "N/D"})
    #elenco_luoghi_attraversati
    container_luoghi_attraversati = wd.find_elements_by_css_selector("ol.luoghi_attraversati li")
    for luogo in range(len(container_luoghi_attraversati)):
        tappa_intermedia = wd.find_elements_by_css_selector("ol.luoghi_attraversati li")[luogo].text
        tappa_intermedia_lat = wd.find_elements_by_css_selector("ol.luoghi_attraversati li")[luogo].get_attribute("data-lat")
        tappa_intermedia_lng = wd.find_elements_by_css_selector("ol.luoghi_attraversati li")[luogo].get_attribute("data-lng")
        elenco_luoghi_attraversati.append({'_id': percorso_id,
                                      'tappa_intermedia': tappa_intermedia,
                                      'tappa_intermedia_lat': tappa_intermedia_lat,
                                      'tappa_intermedia_lng': tappa_intermedia_lng})
#pprint.pprint(elenco_luoghi_attraversati)
ds_info_aggiuntive = pd.DataFrame(info_aggiuntive)
ds_lista_commenti = pd.DataFrame(lista_commenti)
ds_elenco_luoghi_attraversati = pd.DataFrame(elenco_luoghi_attraversati)
ds_elenco_img_altimetrie = pd.DataFrame(elenco_img_altimetrie)

In [ ]:
ds_info_aggiuntive.to_csv('/ds_info_aggiuntive.csv')
ds_lista_commenti.to_csv('/ds_lista_commenti.csv')
ds_elenco_luoghi_attraversati.to_csv('/ds_elenco_luoghi_attraversati.csv')
ds_elenco_img_altimetrie.to_csv('/ds_elenco_img_altimetrie.csv')

In [ ]:
#ho sbagliato assegnazione id nei record del ds_elenco_img_altimetrie, aggiungo colonne in due step [:2000] e [2001:]
ds_elenco_img_altimetrie = pd.read_csv('/content/ds_elenco_img_altimetrie_2001.csv')
ds_dettagli_percorsi = pd.read_csv('/content/ds_dettagli_percorsi.csv')
new_id = []
new_nome_imm = []
for _id, percorso in tqdm(ds_dettagli_percorsi.loc[2001:].iterrows()): #scraping in 2parti
    _id = percorso['_id']
    nome_imm = 'altimetria_' + str(_id) + '.jpg'
    new_id.append(_id)
    new_nome_imm.append(nome_imm)
ds_elenco_img_altimetrie['new_id'] = new_id
ds_elenco_img_altimetrie['new_nome_imm'] = new_nome_imm
ds_elenco_img_altimetrie.to_csv('/ds_elenco_img_altimetrie.csv')

In [10]:
import zipfile
with zipfile.ZipFile('/content/immagini.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/immagini/')

In [ ]:
#ds_dettagli_percorsi = pd.read_csv('/content/ds_dettagli_percorsi.csv')

#rinomina immagini
import os

for _id, percorso in tqdm(ds_dettagli_percorsi.loc[2001:].iterrows()): #scraping in 2parti
    percorso_id = percorso['_id']
    os.rename(r'/content/immagini(1)/altimetria_' + str(_id) + '.jpg', r'/content/immagini(1)/altimetria_' + str(_id) + '_' + str(percorso_id) + '.jpg')
    #print(_id, percorso_id)

In [ ]:
import shutil
zip_name = 'una_gioia'
directory_name = '/content/immagini(1)/'
# Create 'path\to\zip_file.zip'
shutil.make_archive('immagini_2001', 'zip', '/content/immaginin(1)/')

In [ ]:
#ds_info_aggiuntive.info()
#ds_info_aggiuntive.head()
#ds_lista_commenti.info()
#ds_lista_commenti
#ds_elenco_luoghi_attraversati.info()
ds_elenco_luoghi_attraversati

In [ ]:
ds_info_aggiuntive.head()

In [ ]:
ds_info_aggiuntive = pd.read_csv('/content/ds_info_aggiuntive_2001.csv')

In [ ]:
elenco_img_mappa_percorsi = []
for _id, percorso in tqdm(ds_info_aggiuntive.iterrows()): 
    #time.sleep(1 + np.random.randn()*0.2)
    link = percorso["link_imm_percorso"]
    percorso_id = percorso["_id"]
    wd.get(link)
    
    #immagini_altimetrie
    wd.execute_script("window.scrollBy(0,120)")
    wd.save_screenshot("/content/images_mappe_percorsi_1/mappa_percorso_" + str(percorso_id) + ".png") #crea cartella
    
    elenco_img_mappa_percorsi.append({"_id": percorso_id, 
                                      "img_file": "mappa_percorso_" + str(percorso_id) + ".png"})

ds_elenco_img_mappa_percorsi = pd.DataFrame(elenco_img_mappa_percorsi)
ds_elenco_img_mappa_percorsi.to_csv('/ds_elenco_img_mappa_percorsi_2001.csv')

In [ ]:
import shutil
zip_name = 'una_gioia'
directory_name = '/content/images_mappe_percorsi_1/'

# Create 'path\to\zip_file.zip'
shutil.make_archive('mappe_percorsi_2001', 'zip', '/content/images_mappe_percorsi_1/')

'/content/mappe_percorsi_2001.zip'

In [ ]:
"""ds_percorsi_molise_tappe_intermedie = pd.DataFrame(elenco_luoghi_attraversati)
ds_percorsi_molise_tappe_intermedie.set_index("_id")
ds_percorsi_molise_tappe_intermedie.head()"""

In [ ]:
"""ds_percorsi_molise_tappe_intermedie.to_csv('/ds_percorsi_molise_tappe_intermedie.csv')
ds_percorsi_molise_tappe_intermedie = pd.read_csv("/ds_percorsi_molise_tappe_intermedie.csv", index_col="_id")
ds_percorsi_molise_tappe_intermedie.head()

In [ ]:
!pip install clarifai

     |████████████████████████████████| 133kB 2.8MB/s 
  Created wheel for clarifai: filename=clarifai-2.6.2-cp36-none-any.whl size=188404 sha256=7ac740a08ca675d4effe0c0e9c1c4b80a7748318294471925944a2066aebd9be
  Stored in directory: /root/.cache/pip/wheels/17/2c/3b/d89ffd4572633f70872165cedf61813ac8047461ed5e1e3768
Successfully built clarifai


In [ ]:
from clarifai.rest import ClarifaiApp
clarifai_key = "da711b05799b4db697c536dfa65459c9"
app = ClarifaiApp(api_key=clarifai_key)
# use the general model
model = app.public_models.general_model